# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# creat dic for issu num
issu_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
}

In [7]:
# add the issue num colummn to the date frame 
df["issu_num"] = df["issue_d"].apply(lambda x: issu_num[x])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issu_num
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,3
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,3
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,3
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,3
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,3


In [8]:
# next payment date dictinoary 
next_pymnt_num = {
    "Apr-2019": 4,
    "May-2019": 5,
}
   

In [9]:
#add the pamyment column to the data frame 
df["next_pymnt_num"] = df["next_pymnt_d"].apply(lambda x:next_pymnt_num [x])
df.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issu_num,next_pymnt_num
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N,1,5
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N,1,5
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N,1,5
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N,1,5
68816,16000.0,0.1131,350.36,MORTGAGE,72000.0,Verified,Jan-2019,low_risk,n,7.02,...,1.0,0.0,251486.0,74835.0,23000.0,63090.0,N,N,1,5


In [10]:
# Dropping issu_dand next_pymnt_d columns
df.drop(["issue_d","next_pymnt_d"],axis=1,inplace=True)
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issu_num,next_pymnt_num
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,3,5
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,3,5
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,3,5
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,3,5
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,3,5


In [11]:
# encode the column to convert it to numeric format 
df_binary_encoded=pd.get_dummies(df,columns=["home_ownership","verification_status","loan_status","application_type"])
df_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status_high_risk,loan_status_low_risk,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,n,27.24,0.0,0.0,8.0,0.0,...,0,0,1,0,1,0,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,n,20.23,0.0,0.0,17.0,1.0,...,1,0,0,0,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,n,24.26,0.0,0.0,8.0,0.0,...,1,0,0,0,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,n,31.44,0.0,1.0,10.0,1.0,...,0,0,1,0,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,n,18.76,0.0,1.0,14.0,0.0,...,1,0,0,1,0,0,0,1,1,0


In [12]:
# Saving the encoded dataset
file_path = Path("Resources/credit_risk_encoded.csv")
df_binary_encoded.to_csv(file_path, index=False)

## Split the Data into Training and Testing

In [17]:
# Create our features
X =X = df.drop(columns="loan_status")
X=pd.get_dummies(X)

# Create our target
y = y = df["loan_status"]
#y=y.values.reshape(-1, 1)

In [18]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.478007,0.373992,0.148001,1.0,0.123879,0.876121,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499520,0.483865,0.355104,0.0,0.329446,0.329446,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [19]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [20]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51612, 92)

In [21]:
Counter(y_train)

Counter({'low_risk': 51352, 'high_risk': 260})

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [22]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [23]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [24]:
# Scale the training and testing data
X_train_scaled=X_scaler.transform(X_train)
X_test_scaled=X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [25]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_classes=2,
                           n_informative=3, weights=[0.2, 0.8],
                           random_state=1)
clf = BalancedRandomForestClassifier(max_depth=2, random_state=1)
clf.fit(X_train, y_train)  


BalancedRandomForestClassifier(max_depth=2, random_state=1)

In [26]:
# Calculated the balanced accuracy score
y_pred = clf.predict(X_test)
print(y_pred)

['low_risk' 'low_risk' 'low_risk' ... 'low_risk' 'low_risk' 'low_risk']


In [ ]:
accl_score = accuracy_score(y_test, y_pred)
print(accl_score)

In [29]:
# Display the confusion matrix
print(confusion_matrix(y_test, y_pred))

[[   38    49]
 [  330 16788]]


In [ ]:
# Print the imbalanced classification report
print(classification_report(y_test, y_pred))

In [33]:
# List the features sorted in descending order by feature importance
sorted(zip(clf.feature_importances_), reverse=True)

[(0.1686450475326146,),
 (0.12794673232393788,),
 (0.12759598647400625,),
 (0.11046466141498129,),
 (0.08143531359779682,),
 (0.056213387302695955,),
 (0.01839669652439182,),
 (0.017585275261310068,),
 (0.016279869374739454,),
 (0.015616893924081367,),
 (0.014759475712429077,),
 (0.01451784822618881,),
 (0.013623964350500183,),
 (0.01329824981852585,),
 (0.011943944359282845,),
 (0.01102241493140835,),
 (0.009800923194790733,),
 (0.009345166126834847,),
 (0.00922551375851371,),
 (0.009088060071011756,),
 (0.008325988126543678,),
 (0.00829595530910764,),
 (0.00699942382216889,),
 (0.006725911226676906,),
 (0.006072849809671001,),
 (0.005789964031669849,),
 (0.005304675509686509,),
 (0.0050383635875863655,),
 (0.005015993392548898,),
 (0.004760403295229977,),
 (0.004739000704089642,),
 (0.004737335991909273,),
 (0.00441361370792004,),
 (0.004345471146985624,),
 (0.0043129116800731445,),
 (0.004305619640163401,),
 (0.004232122509517883,),
 (0.004026330123441129,),
 (0.0038906894382431615,

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.